# Table 1: Cross-validate model parameter values

In [1]:
import os
import pickle

import numpy as np
import pandas as pd

import argus_shapes

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-colorblind')

/home/mbeyeler/anaconda3/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)
2018-08-04 17:33:30,847 [pulse2percept] [INFO] Welcome to pulse2percept


In [2]:
data_dir = os.environ['ARGUS_SHAPES_DATA']
results_dir = '../results/trial1cv'
col_score = 'best_train_score'
col_groupby = ['subject', 'modelname', 'idx_fold']

In [3]:
subjectdata = argus_shapes.load_subjects(os.path.join(data_dir, 'subjects.csv'))
subjectdata['scale'] = pd.Series([0.75, 0.75, 1.5, 0.5], index=subjectdata.index)
subjects = subjectdata.index.sort_values()

In [4]:
# After repeatedely fitting the model, find the runs with the best training score:
files = argus_shapes.extract_best_pickle_files(results_dir, col_score, col_groupby)

In [5]:
y_true = []
y_pred = []
for file in files:
    yt, yp, best_params, specifics = pickle.load(open(file, 'rb'))
    XX, yy = argus_shapes.calc_mean_images(yt[0].drop(columns='image'), 
                                           yt[0][['electrode', 'image']],
                                           thresh=False, max_area=np.inf,
                                           groupcols=['electrode'])
    yy['subject'] = specifics['subject']
    yy['modelname'] = '0'
    yy['rho'] = best_params[0]['rho']
    if 'axlambda' in best_params[0]:
        yy['axlambda'] = best_params[0]['axlambda']
    if len(XX) > 0:
        yy['img_shape'] = XX['img_shape']
    y_true.append(yy)
    XX, yy = argus_shapes.calc_mean_images(yp[0].drop(columns='image'), 
                                           yp[0][['electrode', 'image']], 
                                           thresh=False, max_area=np.inf,
                                           groupcols=['electrode'])
    yy['subject'] = specifics['subject']
    yy['modelname'] = specifics['modelname']
    yy['rho'] = best_params[0]['rho']
    if 'axlambda' in best_params[0]:
        yy['axlambda'] = best_params[0]['axlambda']
    if len(XX) > 0:
        yy['img_shape'] = XX['img_shape']
    y_pred.append(yy)

In [6]:
y_true = pd.concat(y_true).drop(columns=['x_center', 'y_center'])
y_pred = pd.concat(y_pred).drop(columns=['x_center', 'y_center'])

In [7]:
col_groupby = ['modelname', 'subject', 'electrode']
mu_electrode = y_pred.drop(columns='image').groupby(col_groupby).mean()
mu_electrode.groupby(col_groupby[:2])[['axlambda', 'rho']].agg(['mean', 'sem'])

axlambda                     rho           
                          mean         sem        mean        sem
modelname subject                                                
A         S1               NaN         NaN  532.674499  10.660461
          S2               NaN         NaN  243.824212  33.815198
          S3               NaN         NaN  170.287357   1.163572
          S4               NaN         NaN  174.786790   1.414194
C         S1       1189.556762  156.637505  409.859708   4.697559
          S2        499.679555  141.643332  315.246876  17.074557
          S3        992.265996  149.975085   85.520528   3.087391
          S4       1419.558295   42.481071  437.193516   6.313911